<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Decisions with Multi-AI Agent Personas</span>
</div>

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [2]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.5"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.5"

//using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.5 Microsoft.Extensions.Configuration.Json, 9.0.5

In [3]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.55" 
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.55"
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.55"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.55.0 Microsoft.SemanticKernel.Agents.Abstractions, 1.55.0 Microsoft.SemanticKernel.Agents.Core, 1.55.0

### Step 2 - Decisions with Multi-Agent Personas 

> "The many are smarter than the few when it comes to solving problems, predicting outcomes, and innovating solutions, as long as there is diversity of opinion, independence, and decentralization."  
>
> -- <cite>James Surowiecki (Author "The Wisdom of Crowds")</cite> 

Single agent personas are effective constructs in Semantic Kernel, but can largely be replicated in other mechanisms with prompts. **Creating multiple personas (experts) that can interact with each other is the true power of the Semantic Kernel Agent functionality. This allows multiple chat agents to interact in various ways.** 

Execute the cell below to illustrate how two different personas can interact in a decision-making process moderated by a decision intelligence expert that makes the final reasoning conclusiion on the decision task at hand. In the output notice key steps:
* Each expert (Dwight and Ben) make their recommendation, based on their decision persona
* The final recommendation is made based on the synthesis of the agent recommendations  
* This multi-agent interaction to converge on a single decision mimic the self-consistency GenAI pattern 

Note: The execution below can take the some time. Use GPT-4.1-mini to optimize speed of the execution.  

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Threading;

#pragma warning disable SKEXP0110
// Create a new chat TerminationStrategy that will terminate when the final message contains the term "final decision"
private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("final decision", StringComparison.OrdinalIgnoreCase) ?? false);
}

var assistantDwightEisenhower = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "DwightEisenhower", // Dwight Eisenhower persona
    Description = "A decision intelligence agent that uses the Eisenhower Decision Priorotization Matrix.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Dwight Eisenhower's Decision Prioritization Matrix. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantBenFranklin = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "BenFranklin", // Ben Franklin persona
    Description = "A decision intelligence agent that uses the Ben Franklin Decision Framework.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Ben Franklin's Decision Framework. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantDecisionMaker = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "DecisionMaker",
    Description = "A decision intelligence agent that approves the final decision.",
    Instructions = """
    You are a Decision Intelligence maker. Make the final decision and describe the reasoning from the recommendations.
    You're laser focused on the goal at hand. Don't waste time with chit chat.
    Consider suggestions when refining an idea. 
    When ready to approve the final the decision, use the words final decision to approve the final decision.
    """
};

// Create a chat for agent interaction.
AgentGroupChat chat =
    new(assistantDwightEisenhower, assistantBenFranklin, assistantDecisionMaker)
    {
        ExecutionSettings =
            new()
            {
                // Here a TerminationStrategy subclass is used that will terminate when
                // an assistant message contains the term "approve".
                TerminationStrategy =
                    new ApprovalTerminationStrategy()
                    {
                        // Only the art-director may approve.
                        Agents = [assistantDecisionMaker],
                        // Limit total number of turns
                        MaximumIterations = 10,
                    }
            }
    };

// Invoke chat and display messages.
ChatMessageContent decisionRecommendation = new(AuthorRole.User, """
### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoing. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""");
chat.AddChatMessage(decisionRecommendation);
decisionRecommendation.Content.DisplayAs("text/markdown");
Console.WriteLine();
Console.WriteLine();

await foreach (ChatMessageContent response in chat.InvokeAsync())
{
    response.Content.DisplayAs("text/markdown");
}

Error: (46,1): error SKEXP0110: 'Microsoft.SemanticKernel.Agents.AgentGroupChat' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
(9,52): error SKEXP0110: 'Microsoft.SemanticKernel.Agents.Chat.TerminationStrategy' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

### Step 3 - Advanced Multi-Agent Decision Orchestration  

In [ ]:
// Import the new Magentic orchestration package and Semantic Kernel InProcessRuntime
#r "nuget: Microsoft.SemanticKernel.Agents.Magentic, 1.55.0-preview" 
#r "nuget: Microsoft.SemanticKernel.Agents.Orchestration, 1.55.0-preview" 
#r "nuget: Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.55.0-preview" 

using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Magentic;
using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;

// Create a Devil's advocate agent that rigorously tests and challenges recommendations
var assistantDevilsAdvocate = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "DevilsAdvocate",
    Description = "A decision intelligence agent that acts as a devil's advocate, rigorously testing and challenging recommendations.",
    Instructions = @"
You are a devil’s advocate decision intelligence strategist. 
Your job is to critically evaluate any proposed recommendations or decisions by:
  1. Identifying hidden assumptions and blind spots  
  2. Raising potential risks, unintended consequences, and edge cases  
  3. Questioning feasibility, resource constraints, and stakeholder buy-in  
  4. Suggesting alternative viewpoints or counter-arguments  
  5. Prompting further data, evidence, or analysis where needed  
Respond in a structured way: state each challenge clearly, explain why it matters, 
and recommend how to address or investigate it further."
};


#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001
var magenticManager = new StandardMagenticManager(
    semanticKernel.GetRequiredService<IChatCompletionService>(),
    new OpenAIPromptExecutionSettings())
{
    MaximumInvocationCount = 10, // Maximum number of invocations per orchestration
};


// Initalize the ChatHistory to store the messages exchanged during the orchestration
ChatHistory history = [];

// Define a response callback to handle the responses from the orchestration
// All the agent messages will be added in the history
ValueTask responseCallback(ChatMessageContent response)
{
    history.Add(response);
    return ValueTask.CompletedTask;
}

// Create the MagenticOrchestration with the agents and the response callback
MagenticOrchestration orchestration = new MagenticOrchestration(
    magenticManager,
    assistantDevilsAdvocate,
    assistantDwightEisenhower,
    assistantBenFranklin,
    assistantDecisionMaker)
{
    ResponseCallback = responseCallback,
};

// Create an Semantic Kernel InProcessRuntime to run the orchestration
InProcessRuntime runtime = new InProcessRuntime();
await runtime.StartAsync();

string decisionToProcess = """
### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoing. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";
decisionToProcess.DisplayAs("text/markdown");
Console.WriteLine("----------------------------------------------------------");
Console.WriteLine("----------------------------------------------------------");
"### FINAL RECOMMENDATION".DisplayAs("text/markdown");

// Invoke the orchestration with the decision to process
var orchestrationResult = await orchestration.InvokeAsync(decisionToProcess, runtime);
// Display the decision recommendation from the orchestration result with a timeout of 300 seconds
string decisionRecommendation = await orchestrationResult.GetValueAsync(TimeSpan.FromSeconds(300));
decisionRecommendation.DisplayAs("text/markdown");

Console.WriteLine("----------------------------------------------------------");
Console.WriteLine("----------------------------------------------------------");
"### ORCHESTRATION HISTORY".DisplayAs("text/markdown");

// Show the history of messages exchanged during the orchestration
foreach (ChatMessageContent message in history)
{
    var messageString = $"""
    ### {message.Role} - {message.AuthorName}: 
    {message.Content}
    """;
    messageString.DisplayAs("text/markdown");
}

// Wait for the runtime to complete all tasks
// Dispose the runtime to clean up resources
await runtime.RunUntilIdleAsync();
await runtime.DisposeAsync()

Installed Packages Microsoft.SemanticKernel.Agents.Magentic, 1.55.0-preview Microsoft.SemanticKernel.Agents.Orchestration, 1.55.0-preview Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.55.0-preview

### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoing. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  

----------------------------------------------------------
----------------------------------------------------------


### FINAL RECOMMENDATION